In [1]:
import math
import itertools
import numpy as np

import hoomd
import gsd.hoomd
import coxeter

#### parameters

In [2]:
# integration timestep
dt=0.01

# coupling
kT=1.0
tau=100*dt

#### initialize snapshot

In [3]:
device = hoomd.device.CPU()
sim = hoomd.Simulation(device=device, seed=910)

sim.create_state_from_gsd(filename='./DATA/lattice.gsd')
sim.state.thermalize_particle_momenta(filter=hoomd.filter.All(), 
                                      kT=kT)

#### potential

In [4]:
# neighboring list
nl = hoomd.md.nlist.Cell(buffer=0.4)

# potential
lj = hoomd.md.pair.LJ(nlist=nl)
lj.params[('A', 'A')] = dict(epsilon=2.0, sigma=1.0)
lj.r_cut[('A', 'A')] = 3.0

#### thermodynamic ensemble

In [5]:
nvt = hoomd.md.methods.NVT(filter=hoomd.filter.All(), 
                           kT=kT, 
                           tau=tau)

#### integrator

In [6]:
integrator = hoomd.md.Integrator(dt=dt, 
                                 methods=[nvt],
                                 forces=[lj])

#### writer

In [7]:
thermodynamic_properties = hoomd.md.compute.ThermodynamicQuantities(filter=hoomd.filter.All())

logger = hoomd.logging.Logger()
logger.add(sim, quantities=['timestep', 'walltime'])
logger.add(thermodynamic_properties)

gsd_writer = hoomd.write.GSD(filename='./DATA/randomized.gsd',
                             trigger=hoomd.trigger.Periodic(100),
                             log=logger,
                             mode='wb')

#### attaching operations

In [9]:
sim.operations.writers.append(gsd_writer)
sim.operations.integrator = integrator
sim.operations.computes.append(thermodynamic_properties)

#### run

In [10]:
sim.run(1e6)